In [44]:
using CSV, DataFrames, Turing, CategoricalArrays, StatsBase, StatsPlots, Random,
    ReverseDiff, Revise, RCall, LinearAlgebra
using OptimizationOptimJL, Distributions, ApproxFun, Serialization, Printf, DataFramesMeta,
    StatProfilerHTML, StatsFuns, OptimizationBBO, Printf

using Interact, Loess
includet("debughelpers.jl")
includet("fithelpers.jl")
includet("gen_inits.jl")
includet("models.jl")
includet("fitmodel1.jl")
includet("fitmodel2.jl")
## includet("fitmodel3.jl")


In [2]:
## Load the data and create the model object

##Random.seed!(20240719)


#optis = CSV.read("./fitted_models/opti" * model * ".csv", DataFrame)
optis = nothing

dt = load_flows()
dt.fromdist = categorical(dt.fromdist)
dt.todist = categorical(dt.todist)
dt.agegroup = categorical(dt.agegroup)
levels!(dt.agegroup,["below18","18-25","25-30","30-50","50-65","above65"])
rename!(dt, Dict(:dist => :distance))

## Create a districts file which has distcode, pop, density, xcoord, ycoord and save it in the data directory
dists = CSV.read("./data/districts.csv",DataFrame)
dists.distcode = categorical(dists.distcode)

distances = [norm([dists.xcoord[i]-dists.xcoord[j],dists.ycoord[i]-dists.ycoord[j]]) for i in 1:nrow(dists) , j in 1:nrow(dists) if i != j]
meddist = median(distances)
distances = nothing ## free the memory

popgerm = sum(dists.pop)

Nages = length(levels(dt.agegroup))
Ndist = nrow(dists)

netactual = calcnet(dt.flows,
            levelcode.(dt.fromdist),
            levelcode.(dt.todist),
            levelcode.(dt.agegroup),
            Nages,
            Ndist)
Ncoefs = 36

dtsmall = dt[rand(Bernoulli(.02),nrow(dt)),:]
dtsmall = dtsmall[dtsmall.flows .> 0.0,:]

model3 = migration3(dtsmall.flows,sum(dtsmall.flows),levelcode.(dtsmall.fromdist),levelcode.(dtsmall.todist),
                        dtsmall.frompop,dtsmall.topop, popgerm, dtsmall.distance, levelcode.(dtsmall.agegroup),Nages,
                        dists.xcoord,dists.ycoord,dists.density,dists.pop,Ndist,meddist,netactual,Ncoefs)



DynamicPPL.Model{typeof(migration3), (:flows, :allmoves, :fromdist, :todist, :frompop, :topop, :popgerm, :distance, :agegroup, :Nages, :xcoord, :ycoord, :density, :distpop, :Ndist, :meddist, :netactual, :ncoefs), (), (), Tuple{Vector{Int64}, Int64, Vector{Int64}, Vector{Int64}, Vector{Float64}, Vector{Float64}, Float64, Vector{Int64}, Vector{Int64}, Int64, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Int64, Float64, Matrix{Int64}, Int64}, Tuple{}, DynamicPPL.DefaultContext}(migration3, (flows = [14, 1, 2, 6, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 2, 4, 1], allmoves = 43876, fromdist = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  401, 401, 401, 401, 401, 401, 401, 401, 401, 401], todist = [2, 10, 19, 30, 58, 82, 133, 149, 260, 336  …  51, 88, 184, 191, 199, 233, 346, 350, 361, 375], frompop = [11.765, 16.959, 17.061, 8.839, 8.839, 17.061, 17.061, 8.839, 6.346, 15.804  …  3.275, 23.222, 3.275, 19.435, 19.435, 19.435, 3.275, 19.435, 3.217, 3.217], topop = [32.068, 32.917, 2

# Find Inits for model3

The goal will be to load in the data, and find the initial conditions for model 3 that are a good starting point in terms of the parameters
* a
* c
* d0
* dscale
* logisticconst

In [43]:


@manipulate for a1 = -10.0:.1:10.0,
                a2 = -10.0:.1:10.0,
                a3 = -10.0:.1:10.0,
                a4 = -10.0:.1:10.0,
                a5 = -10.0:.1:10.0,
                a6 = -10.0:.1:10.0,
                c = 0.01:.1:10.0,
                d0 = 0.001:.01:2.0,
                dscale = 0.05:.05:2.5,
                neterr = 0.1:.1:2.0,
                logisticconst = -10:.1:10
print(d0)
    Nages = 6
    ncoefs = 36
    as = [.7, 2.3, 2.5, .1, -.3, -.3]
   ##as = [-10, -10, -10, -10, -10, -.3] 
    cs = [2.5, 2.5, 2.5, 2.5, 2.5, 2.5]
    d0s = [.1, .1, .1, .1, .1, .1]
    d0s = fill(0.0, Nages);
    dscales = [.3, .3, .5, .5, .3, .3]
    dscales = fill(0.5, Nages);
    logconst = -5
   ##as = fill(-2, Nages)
   logconst = 4.5
    vals = [as;
            cs;
            d0s;   
            dscales;
            [neterr, logconst];
            fill(0.0,Nages); #kd
            fill(0.0,Nages*Ncoefs)
            ]
    
    names = [["a[$i]" for i in 1:Nages];
             ["c[$i]" for i in 1:Nages];
             ["d0[$i]" for i in 1:Nages];
             ["dscale[$i]" for i in 1:Nages];
             ["neterr","logisticconst"];
             ["kd[$i]" for i in 1:Nages];
             ["desirecoefs[$i]" for i in 1:(Ncoefs*Nages)];
             ]
    
    (preds,netflows) = generated_quantities(model3,vals,names)
    #@show preds
    scatter(dtsmall.distance, log.(dtsmall.flows ./ preds),group=dtsmall.agegroup,alpha=0.2,ylim=(-4,4))
end


0.991

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["a1"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 201, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(101), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/donkon/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/donkon/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/donkon/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/home/donkon/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/home/donkon/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x00007fe4eaba95f0, Task (runnable) @0x00007fe4eaba95f0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 140627669169904, 144701614814)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-10.0\",\"-9.9\",\"-9.8\",\"-9.7\",\"-9.6\",\"-9.5\",\"-9.4\",\"-9.3\",\"-9.2\",\"-9.1\",\"-9.0\",\"-8.9\",\"-8.8\",\"-8.7\",\"-8.6\",\

In [4]:
Nages = 
        vals = [[a1,a2,a3,a4,a5,a6];
            fill(c,Nages);
            fill(d0,Nages);
            fill(dscale,Nages);
            [neterr, logisticconst];
            fill(0.0,Nages); #kd
            fill(0.0,Nages*Ncoefs)
            ]
    names = [["a[$i]" for i in 1:Nages];
             ["c[$i]" for i in 1:Nages];
             ["d0[$i]" for i in 1:Nages];
             ["dscale[$i]" for i in 1:Nages];
             ["neterr","logisticconst"];
             ["kd[$i]" for i in 1:Nages];
             ["desirecoefs[$i]" for i in 1:(Ncoefs*Nages)];
             ]
    
    (preds,netflows) = generated_quantities(model3,vals,names)
    #@show preds
    scatter(dtsmall.distance, log.(dtsmall.flows ./ preds),group=dtsmall.agegroup,alpha=0.2,ylim=(-4,4))
end


LoadError: MethodError: no method matching (::Colon)(::Int64, ::Vector{Any})

[0mClosest candidates are:
[0m  (::Colon)(::Real, ::Any, [91m::Union{Infinities.InfiniteCardinal{0}, Infinities.Infinity}[39m)
[0m[90m   @[39m [35mInfiniteArrays[39m [90m~/.julia/packages/InfiniteArrays/yaXl7/src/[39m[90m[4minfrange.jl:12[24m[39m
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Unitful.Quantity}
[0m[90m   @[39m [32mUnitful[39m [90m~/.julia/packages/Unitful/GYzMo/src/[39m[90m[4mrange.jl:86[24m[39m
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:50[24m[39m
[0m  ...
